<a href="https://colab.research.google.com/github/pablogadhi/TranslationDetector/blob/master/sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/pablogadhi/TranslationDetector
!python -m spacy download en
!python -m spacy download es
%cd TranslationDetector
!pip install --upgrade torch torchtext

fatal: destination path 'TranslationDetector' already exists and is not an empty directory.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/es_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/es
You can now load the model via spacy.load('es')
/content/TranslationDetector
Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.5.0+cu101)
Requirement already up-to-date: torchtext in /usr/local/lib/python3.6/dist-packages (0.6.0)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!git pull

Already up to date.


In [0]:
import torch
from translation.train import train_model
from translation.data_loader import load_data, make_iters
from translation.transformer import Transformer

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
!nvidia-smi

Wed Jun 10 07:54:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    86W / 149W |     11MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### Load data and create the model

In [0]:
DATA_DIR='/content/drive/My Drive/NLP/data/'

In [0]:
SRC, TGT, train, valid, test = load_data(
        DATA_DIR + "en-es_", "en.txt", "es.txt")

In [0]:
train_itr, valid_itr, test_itr = make_iters(
        train, valid, test, device, batch_size=4000)
model = Transformer(len(SRC.vocab), len(TGT.vocab)).to(device)

In [10]:
len(SRC.vocab.itos)

40002

### Train the model

In [0]:
train_model(model, train_itr, valid_itr, SRC, TGT, device, 10, save_at=1, checkpoint_f=DATA_DIR + "checkpoint.pt")

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Step    50 : 1263.67 ms/batch | loss  7.94
Step   100 : 1094.34 ms/batch | loss  7.11
Step   150 : 1123.40 ms/batch | loss  5.87
Step   200 : 1117.22 ms/batch | loss  5.47
Step   250 : 1117.48 ms/batch | loss  5.13
Step   300 : 1110.92 ms/batch | loss  5.08
Step   350 : 1126.74 ms/batch | loss  4.99
Step   400 : 1123.65 ms/batch | loss  4.83
Step   450 : 1114.03 ms/batch | loss  5.00
Step   500 : 1124.16 ms/batch | loss  4.70
Step   550 : 1119.27 ms/batch | loss  4.49
Step   600 : 1117.54 ms/batch | loss  4.73
Step   650 : 1119.80 ms/batch | loss  4.55
Step   700 : 1114.66 ms/batch | loss  4.33
Step   750 : 1132.81 ms/batch | loss  4.45
